In [ ]:
import os
import subprocess
import time


In [ ]:
def burn_subtitles(source_dir, destination_dir):
    # Define the range of seasons
    seasons = sorted([f"Season {i}" for i in range(1, 8)])
    
    # Flag to track if .mkv.part files are found
    mkv_part_found = True

    # Continue running the loop until no .mkv.part files are found
    while mkv_part_found:
        mkv_part_found = False  # Reset flag

        # Loop through each season folder
        for season in seasons:
            season_path = os.path.join(source_dir, season)
            
            # Check if the season directory exists
            if not os.path.isdir(season_path):
                print(f"Directory {season_path} does not exist, skipping.")
                continue

            # Create corresponding destination directory for the season
            destination_season_path = os.path.join(destination_dir, season)
            os.makedirs(destination_season_path, exist_ok=True)

            # Get sorted list of files in the season folder
            files = sorted(os.listdir(season_path))

            # Process each file in the season folder
            for filename in files:
                # Look for complete .mkv files
                if filename.endswith('.mkv'):
                    if filename.endswith('.mkv.part'):
                        mkv_part_found = True  # Found a .mkv.part file
                        continue  # Skip .mkv.part files

                    source_file = os.path.join(season_path, filename)
                    output_file = os.path.join(destination_season_path, filename)

                    # **Check if the output file already exists**
                    if os.path.exists(output_file):
                        print(f"File already exists, skipping: {output_file}")
                        continue  # Skip this file as it's already processed

                    # Command to burn subtitles, re-encode video, and exclude audio
                    ffmpeg_command = [
                        'ffmpeg',
                        '-i', source_file,
                        '-filter_complex', '[0:v][0:s:0]overlay',  # Overlay subtitles
                        '-c:v', 'libx264',      # Re-encode with H.264
                        '-preset', 'veryslow',      # Use veryslow preset for encoding speed
                        '-crf', '18',           # Constant Rate Factor for quality
                        '-g', '48',             # GOP size (keyframe interval)
                        # '-movflags', '+faststart',  # Enable fast start for MP4
                        '-an',                  # Exclude audio
                        output_file
                    ]

                    try:
                        # Execute the FFmpeg command
                        print(f"Processing {filename}...")
                        subprocess.run(ffmpeg_command, check=True)
                        print(f"Subtitled file created: {output_file}")
                    except subprocess.CalledProcessError as e:
                        print(f"Error processing {filename}: {e}")

        # Sleep for a while before rechecking
        if mkv_part_found:
            print("Waiting for .mkv.part files to complete...")
            time.sleep(600)  # Wait for 10 minutes before checking again



In [ ]:
# Specify the source and destination directories
source_directory = "/media/coof/standardHDD/Veep (2012) Season 1-7 S01-S07 (1080p BluRay x265 HEVC 10bit AAC 5.1 Silence)"  # Replace with the path to your directory containing seasons
destination_directory = "./Veep2"

# Call the function to burn subtitles
burn_subtitles(source_directory, destination_directory)
